In [4]:
import numpy as np
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D , MaxPooling2D , Dense , Flatten , Dropout 
from tensorflow.keras.optimizers import SGD
from keras.layers import BatchNormalization

In [3]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=4afafbd548bdf9a2f4e2b919efdc30934201bccc8ed42a10b35b441328d839f7
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built gputil
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Gen RAM Free: 12.4 GB  | Proc size: 544.0 MB
GPU RAM Free: 15109MB | Used: 0MB | Util   0% | Total 15109MB


In [5]:
(x_train , y_train) , (x_test , y_test) = cifar10.load_data()

170508288/170498071 [==============================] - 4s 0us/step


In [6]:
# One Hot Encoding
# print(np.unique(y_train))

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


In [7]:
# Normalization
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train/=255
x_test/=255

In [ ]:
model = Sequential()

# kernel_initializer = 'he_uniform' . Weights assigned would be uniform

model.add(Conv2D(32 , (3,3) , padding = 'same' , activation = 'relu' , kernel_initializer = 'he_uniform' , input_shape = (32,32,3)))
model.add(Conv2D(32 , (3,3) , padding = 'same' , activation = 'relu' , kernel_initializer = 'he_uniform'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(128 , activation = 'relu' , kernel_initializer = 'he_uniform'))
model.add(Dense(10 , activation = 'softmax'))

print(model.summary())



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 conv2d_3 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dense_2 (Dense)             (None, 128)               1048704   
                                                                 
 dense_3 (Dense)             (None, 10)                1290      
                                                      

In [ ]:
optimizer = SGD(learning_rate = 0.001 , momentum = 0.95)

model.compile(optimizer = optimizer , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
model.fit(x_train , y_train , epochs = 50 , batch_size = 64 , verbose = 1 , validation_data = (x_test , y_test))

Epoch 1/50
782/782 [==============================] - 16s 6ms/step - loss: 1.6493 - accuracy: 0.4102 - val_loss: 1.3781 - val_accuracy: 0.5041
Epoch 2/50
782/782 [==============================] - 5s 6ms/step - loss: 1.3043 - accuracy: 0.5378 - val_loss: 1.2135 - val_accuracy: 0.5766
Epoch 3/50
782/782 [==============================] - 5s 6ms/step - loss: 1.1540 - accuracy: 0.5920 - val_loss: 1.1571 - val_accuracy: 0.5932
Epoch 4/50
782/782 [==============================] - 5s 6ms/step - loss: 1.0411 - accuracy: 0.6331 - val_loss: 1.0937 - val_accuracy: 0.6105
Epoch 5/50
782/782 [==============================] - 5s 6ms/step - loss: 0.9440 - accuracy: 0.6668 - val_loss: 1.0144 - val_accuracy: 0.6438
Epoch 6/50
782/782 [==============================] - 4s 6ms/step - loss: 0.8644 - accuracy: 0.6977 - val_loss: 0.9958 - val_accuracy: 0.6534
Epoch 7/50
782/782 [==============================] - 5s 6ms/step - loss: 0.7920 - accuracy: 0.7262 - val_loss: 0.9670 - val_accuracy: 0.6670
Epoch

In [ ]:
results = model.evaluate(x_test , y_test , verbose = 1)
results

313/313 [==============================] - 1s 3ms/step - loss: 2.4885 - accuracy: 0.6812


[2.488542318344116, 0.6812000274658203]

We get 68% accuracy , so now we will tune the parameters using Regularization (Dropout() and BatchNormalization())

In [9]:
model = Sequential()

# kernel_initializer = 'he_uniform' . Weights assigned would be uniform

model.add(Conv2D(32 , (3,3) , padding = 'same' , activation = 'relu' , kernel_initializer = 'he_uniform' , input_shape = (32,32,3)))
model.add(BatchNormalization())

model.add(Conv2D(32 , (3,3) , padding = 'same' , activation = 'relu' , kernel_initializer = 'he_uniform'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2)) # neurons with 20% probability will be ignored

model.add(Conv2D(32 , (3,3) , padding = 'same' , activation = 'relu' , kernel_initializer = 'he_uniform' , input_shape = (32,32,3)))
model.add(BatchNormalization())

model.add(Conv2D(32 , (3,3) , padding = 'same' , activation = 'relu' , kernel_initializer = 'he_uniform'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2)) # neurons with 20% probability will be ignored

model.add(Conv2D(32 , (3,3) , padding = 'same' , activation = 'relu' , kernel_initializer = 'he_uniform' , input_shape = (32,32,3)))
model.add(BatchNormalization())

model.add(Conv2D(32 , (3,3) , padding = 'same' , activation = 'relu' , kernel_initializer = 'he_uniform'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(128 , activation = 'relu' , kernel_initializer = 'he_uniform'))
model.add(Dropout(0.2))
model.add(Dense(10 , activation = 'softmax'))

optimizer = SGD(learning_rate = 0.001 , momentum = 0.95)

model.compile(optimizer = optimizer , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
model.fit(x_train , y_train , epochs = 50 , batch_size = 64 , verbose = 1 , validation_data = (x_test , y_test))

model.evaluate(x_test , y_test)

Epoch 1/50
782/782 [==============================] - 8s 9ms/step - loss: 1.7761 - accuracy: 0.3594 - val_loss: 1.4028 - val_accuracy: 0.4793
Epoch 2/50
782/782 [==============================] - 7s 8ms/step - loss: 1.4168 - accuracy: 0.4829 - val_loss: 1.3460 - val_accuracy: 0.5142
Epoch 3/50
782/782 [==============================] - 7s 8ms/step - loss: 1.2731 - accuracy: 0.5389 - val_loss: 1.1535 - val_accuracy: 0.5822
Epoch 4/50
782/782 [==============================] - 7s 9ms/step - loss: 1.1680 - accuracy: 0.5779 - val_loss: 1.1380 - val_accuracy: 0.5910
Epoch 5/50
782/782 [==============================] - 7s 8ms/step - loss: 1.0970 - accuracy: 0.6042 - val_loss: 1.0047 - val_accuracy: 0.6457
Epoch 6/50
782/782 [==============================] - 7s 8ms/step - loss: 1.0446 - accuracy: 0.6234 - val_loss: 1.0137 - val_accuracy: 0.6403
Epoch 7/50
782/782 [==============================] - 7s 10ms/step - loss: 0.9949 - accuracy: 0.6418 - val_loss: 0.9618 - val_accuracy: 0.6612
Epoch

[0.5504407286643982, 0.8118000030517578]